In [1]:
# Data tools
import pandas as pd

In [28]:
# Plotting tools
import plotly.graph_objs as go
import plotly.offline as pyoff

In [30]:
pyoff.init_notebook_mode(connected=True)

# Read Data

In [23]:
df = pd.read_csv("../data/free_throws.csv")

Show column datatypes

In [24]:
df.dtypes

end_result     object
game           object
game_id       float64
period        float64
play           object
player         object
playoffs       object
score          object
season         object
shot_made       int64
time           object
dtype: object

Show the dataframe

In [25]:
df

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18
5,106 - 114,PHX - LAL,261031013.0,1.0,Amare Stoudemire makes free throw 1 of 2,Amare Stoudemire,regular,33 - 20,2006 - 2007,1,3:15
6,106 - 114,PHX - LAL,261031013.0,1.0,Amare Stoudemire makes free throw 2 of 2,Amare Stoudemire,regular,34 - 20,2006 - 2007,1,3:15
7,106 - 114,PHX - LAL,261031013.0,2.0,Leandro Barbosa misses free throw 1 of 2,Leandro Barbosa,regular,43 - 29,2006 - 2007,0,10:52
8,106 - 114,PHX - LAL,261031013.0,2.0,Leandro Barbosa makes free throw 2 of 2,Leandro Barbosa,regular,44 - 29,2006 - 2007,1,10:52
9,106 - 114,PHX - LAL,261031013.0,2.0,Lamar Odom makes free throw 1 of 2,Lamar Odom,regular,44 - 30,2006 - 2007,1,10:37


## Describe potentially interesting columns

In [17]:
df.player.describe()

count           618019
unique            1098
top       LeBron James
freq              8001
Name: player, dtype: object

In [18]:
df.time.describe()

count     618019
unique       534
top         0:00
freq        2666
Name: time, dtype: object

In [26]:
df.season.describe()

count          618019
unique             10
top       2006 - 2007
freq            67612
Name: season, dtype: object

# Analyse the number of shots made per season

In [35]:
season_shots = df.groupby(df.season)["shot_made"].sum()

In [36]:
season_shots

season
2006 - 2007    50841
2007 - 2008    49494
2008 - 2009    50320
2009 - 2010    49153
2010 - 2011    49013
2011 - 2012    36466
2012 - 2013    43881
2013 - 2014    47127
2014 - 2015    45005
2015 - 2016    46389
Name: shot_made, dtype: int64

In [61]:
# Create dataset
data = [go.Bar(x=season_shots.index, y=season_shots.values)]
# Create graph layout
layout = dict(
    title='Number of shots per season'
)
# Combine the two into a figure and display it
fig = go.Figure(data=data, layout=layout)
pyoff.iplot(fig)